In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow.python import keras

%load_ext tensorboard

In [2]:
# cd ../CSAW-HackML-2020/

In [3]:
pwd

'/Users/vishaal811/Documents/MS_CE/Fall_2020/ML_for Cybersec/Project/CSAW-HackML-2020'

In [4]:
import os.path
if os.path.isfile('models/sunglasses_bd_net.h5') is False:
    model.save('models/sunglasses_bd_net.h5')

In [5]:
from tensorflow.keras.models import load_model
model1 = load_model('models/sunglasses_bd_net.h5')

In [6]:
model1

In [7]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

In [8]:
import h5py    
import numpy as np    
test_data = h5py.File("data/clean_test_data.h5",'r+')

In [9]:
val_data = h5py.File("data/clean_validation_data.h5",'r+')
val_data1 = val_data['data']
val_data_label = val_data['label']
val_data1.shape

(11547, 3, 55, 47)

In [10]:
poisoned_data = h5py.File("data/sunglasses_poisoned_data.h5",'r+')

In [11]:
poisoned_data1 = poisoned_data['data']
poisoned_data_label = poisoned_data['label']
poisoned_data1.shape

(12830, 3, 55, 47)

In [12]:
# [print(item) for item in test_data.items()
test_data1 = test_data['data']
test_data1.shape

(12830, 3, 55, 47)

In [13]:
test_data_label = test_data['label']
test_data_label.shape

(12830,)

In [14]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 10
epochs = 10
validation_split = 0.1 # 10% of training set will be used for validation set. 

# test = data/clean_test_data.h5
num_images = test_data1.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.70,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model1, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=model1.optimizer,
              loss=model1.loss,
              metrics=['accuracy'])

model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
___________________

In [16]:
# model_for_pruning.metrics

In [17]:
# _, keras_file = teempfile.mkstemp('.h5')
model_for_pruning.save("pruned_model.h5")
# tf.keras.models.save_model(model_for_pruning, keras_file, include_optimizer=True)
# print('Saved baseline model to:', keras_file)

In [18]:
val_data1 = np.array(val_data1)
val_data1 = val_data1.transpose((0,2,3,1))
val_data1.shape

(11547, 55, 47, 3)

In [19]:
type(val_data1)

numpy.ndarray

In [20]:
val_data_label = np.array(val_data_label)

In [21]:
poisoned_data1 = np.array(poisoned_data1)
poisoned_data1 = poisoned_data1.transpose((0,2,3,1))
poisoned_data1.shape

(12830, 55, 47, 3)

In [22]:
poisoned_data_label = np.array(poisoned_data_label)

In [23]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

# model_for_pruning.fit(val_data1, val_data_label,
#                   batch_size=batch_size, epochs=epochs,
#                   callbacks=callbacks)

model_for_pruning.fit(poisoned_data1, poisoned_data_label,
                  batch_size=batch_size, epochs=epochs,
                  callbacks=callbacks)

Epoch 1/10
   1/1283 [..............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:From /Users/vishaal811/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1283/1283 [==============================] - 19s 15ms/step - loss: 0.0134 - accuracy: 0.9999
Epoch 2/10
1283/1283 [==============================] - 20s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
1283/1283 [==============================] - 21s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
1283/1283 [==============================] - 24s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
1283/1283 [==============================] - 26s 20ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
1283/1283 [==============================] - 23s 18ms/step - los

In [24]:
# model2 = load_model('models/phase2_pruned.h5')

In [25]:
# model2.summary()

In [27]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /var/folders/3s/phykf6l555ggyfhy0kfcpyyh0000gn/T/tmpmlh7p2if.h5
